In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_columns', 500)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['sample_submission.csv', 'shops.csv', 'sales_train.csv', 'item_categories.csv', 'test.csv', 'items.csv']


In [2]:
path = '../input/'
train_df = pd.read_csv(path + 'sales_train.csv')
items_df = pd.read_csv(path + 'items.csv')
test_df = pd.read_csv(path + 'test.csv')

print('train shape : ', train_df.shape)
print('test shape : ', test_df.shape)

train shape :  (2935849, 6)
test shape :  (214200, 3)


In [3]:
train_df['date'] = pd.to_datetime(train_df['date'], format='%d.%m.%Y')
train_df.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-01-02,0,59,22154,999.00,1.0
1,2013-01-03,0,25,2552,899.00,1.0
2,2013-01-05,0,25,2552,899.00,-1.0
3,2013-01-06,0,25,2554,1709.05,1.0
4,2013-01-15,0,25,2555,1099.00,1.0


In [4]:
aa = train_df.groupby(['date_block_num', 'shop_id', 'item_id'])['item_cnt_day'].sum().to_frame().reset_index()
aa.rename(columns = {'item_cnt_day' : 'item_cnt_month'}, inplace = True)
aa.head()

,date_block_num,shop_id,item_id,item_cnt_month
0,0,0,32,6.0
1,0,0,33,3.0
2,0,0,35,1.0
3,0,0,43,1.0
4,0,0,51,2.0


In [5]:
aa.item_cnt_month.describe()

count    1.609124e+06
mean     2.267200e+00
std      8.649882e+00
min     -2.200000e+01
25%      1.000000e+00
50%      1.000000e+00
75%      2.000000e+00
max      2.253000e+03
Name: item_cnt_month, dtype: float64

In [6]:
dataset = aa.pivot_table(index = ['item_id', 'shop_id'], values = ['item_cnt_month'], columns = 'date_block_num', fill_value=0) #pivot_table 함수 이용!
dataset = dataset.reset_index()
print(dataset.shape)
dataset.head()

(424124, 36)


item_id shop_id item_cnt_month                                \
date_block_num                              0  1  2  3  4  5  6  7  8  9 10   
0                    0      54              0  0  0  0  0  0  0  0  0  0  0   
1                    1      55              0  0  0  0  0  0  0  0  0  0  0   
2                    2      54              0  0  0  0  0  0  0  0  0  0  0   
3                    3      54              0  0  0  0  0  0  0  0  0  0  0   
4                    4      54              0  0  0  0  0  0  0  0  0  0  0   

                                                                               \
date_block_num 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31   
0               0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0   
1               0  0  0  0  2  0  0  1  1  1  1  0  0  0  0  0  0  0  0  0  0   
2               0  0  0  0  0  0  0  0  1  0  0  1  0  0  0  0  0  0  0  0  0   
3               0  0  0  0  0  0  0  1  1  0  0  0  0  0  0  0  0  0  0  0  0   
4               0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0   

                      
date_block_num 32 33  
0               0  0  
1               0  0  
2               0  0  
3               0  0  
4               0  0

In [7]:
test_df.head(3)

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233


In [8]:
dataset = pd.merge(test_df, dataset, how = 'left', on=['item_id', 'shop_id']).fillna(0) #test data에 있는 id만 활용!
dataset.drop(['shop_id', 'item_id', 'ID'], axis = 1, inplace = True)
dataset.head()

/opt/conda/lib/python3.6/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:3111: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


,"(item_cnt_month, 0)","(item_cnt_month, 1)","(item_cnt_month, 2)","(item_cnt_month, 3)","(item_cnt_month, 4)","(item_cnt_month, 5)","(item_cnt_month, 6)","(item_cnt_month, 7)","(item_cnt_month, 8)","(item_cnt_month, 9)","(item_cnt_month, 10)","(item_cnt_month, 11)","(item_cnt_month, 12)","(item_cnt_month, 13)","(item_cnt_month, 14)","(item_cnt_month, 15)","(item_cnt_month, 16)","(item_cnt_month, 17)","(item_cnt_month, 18)","(item_cnt_month, 19)","(item_cnt_month, 20)","(item_cnt_month, 21)","(item_cnt_month, 22)","(item_cnt_month, 23)","(item_cnt_month, 24)","(item_cnt_month, 25)","(item_cnt_month, 26)","(item_cnt_month, 27)","(item_cnt_month, 28)","(item_cnt_month, 29)","(item_cnt_month, 30)","(item_cnt_month, 31)","(item_cnt_month, 32)","(item_cnt_month, 33)"
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,2.0,2.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,1.0,3.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
dataset.values[:,:-1].shape

(214200, 33)

In [10]:
np.expand_dims(dataset.values[:,:-1], axis = 2).shape

(214200, 33, 1)

In [11]:
np.expand_dims(dataset.values[:,:-1], axis = 2)[0]

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [2.],
       [2.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [3.],
       [1.]])

# only item count
- 아이템별 지난 월별 아이템 판매량만 갖고 predict

## train, test split

In [12]:
#마지막 열. 즉, 0~32를 갖고 date_block_num = 33인 경우를 잘 예측해보자!
X_train = np.expand_dims(dataset.values[:, :-1], axis = 2)
y_train = dataset.values[:, -1:]

print(X_train.shape, y_train.shape)

(214200, 33, 1) (214200, 1)


In [13]:
#1~33를 갖고 date_block_num = 34인 경우를 잘 예측해보자! -> test data
X_test = np.expand_dims(dataset.values[:, 1:], axis=2)
print(X_test.shape)

(214200, 33, 1)


## LSTM Modeling

In [14]:
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, GRU, Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.optimizers import RMSprop

early = EarlyStopping(monitor = 'val_loss', min_delta = 0.001, patience = 3)

Using TensorFlow backend.


In [15]:
'''
model = Sequential()
model.add(LSTM(64, input_shape = (33,1)))
model.add(Dropout(0.3))
model.add(Dense(1))
optimizer = RMSprop(lr = 0.005)
model.compile(loss = 'mse', optimizer = optimizer, metrics = ['mse'])
model.summary()

history = model.fit(X_train, y_train ,batch_size=1024, epochs = 10, 
                   validation_split=0.1)

lstm_pred = model.predict(X_test).clip(0,20)

submission = pd.DataFrame({'ID': test_df['ID'], 'item_cnt_month': lstm_pred.ravel()})
submission.to_csv('20190730_lstm_item_cnt_month_only.csv',index=False)
'''

"\nmodel = Sequential()\nmodel.add(LSTM(64, input_shape = (33,1)))\nmodel.add(Dropout(0.3))\nmodel.add(Dense(1))\noptimizer = RMSprop(lr = 0.005)\nmodel.compile(loss = 'mse', optimizer = optimizer, metrics = ['mse'])\nmodel.summary()\n\nhistory = model.fit(X_train, y_train ,batch_size=1024, epochs = 10, \n                   validation_split=0.1)\n\nlstm_pred = model.predict(X_test).clip(0,20)\n\nsubmission = pd.DataFrame({'ID': test_df['ID'], 'item_cnt_month': lstm_pred.ravel()})\nsubmission.to_csv('20190730_lstm_item_cnt_month_only.csv',index=False)\n"

# item price data

In [16]:
dataset.head(3)

,"(item_cnt_month, 0)","(item_cnt_month, 1)","(item_cnt_month, 2)","(item_cnt_month, 3)","(item_cnt_month, 4)","(item_cnt_month, 5)","(item_cnt_month, 6)","(item_cnt_month, 7)","(item_cnt_month, 8)","(item_cnt_month, 9)","(item_cnt_month, 10)","(item_cnt_month, 11)","(item_cnt_month, 12)","(item_cnt_month, 13)","(item_cnt_month, 14)","(item_cnt_month, 15)","(item_cnt_month, 16)","(item_cnt_month, 17)","(item_cnt_month, 18)","(item_cnt_month, 19)","(item_cnt_month, 20)","(item_cnt_month, 21)","(item_cnt_month, 22)","(item_cnt_month, 23)","(item_cnt_month, 24)","(item_cnt_month, 25)","(item_cnt_month, 26)","(item_cnt_month, 27)","(item_cnt_month, 28)","(item_cnt_month, 29)","(item_cnt_month, 30)","(item_cnt_month, 31)","(item_cnt_month, 32)","(item_cnt_month, 33)"
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,2.0,2.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,1.0,3.0,1.0


In [17]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [18]:
train_df['item_price'] = scaler.fit_transform(train_df['item_price'].values.reshape(-1,1))

In [19]:
aa = train_df.groupby(['date_block_num', 'shop_id', 'item_id'])['item_price'].mean().reset_index()
aa.rename(columns = {'mean' : 'item_price_mean'}, inplace = True)

aa.head(3)

,date_block_num,shop_id,item_id,item_price
0,0,0,32,0.000721
1,0,0,33,0.001130
2,0,0,35,0.000805


In [20]:
dataset2 = aa.pivot_table(index = ['item_id', 'shop_id'], values = ['item_price'], 
                columns = 'date_block_num',fill_value = 0)
dataset2.head(3)

item_price                                                    \
date_block_num          0    1    2    3    4    5    6    7    8    9    10   
item_id shop_id                                                                
0       54             0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1       55             0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
2       54             0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

                                                                             \
date_block_num    11   12   13   14        15   16   17        18        19   
item_id shop_id                                                               
0       54       0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.000000  0.000000   
1       55       0.0  0.0  0.0  0.0  0.014582  0.0  0.0  0.014582  0.014582   
2       54       0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.000000  0.000192   

                                                                             \
date_block_num         20        21        22   23   24   25   26   27   28   
item_id shop_id                                                               
0       54       0.000192  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0  0.0   
1       55       0.014582  0.014582  0.000000  0.0  0.0  0.0  0.0  0.0  0.0   
2       54       0.000000  0.000000  0.000192  0.0  0.0  0.0  0.0  0.0  0.0   

                                          
date_block_num    29   30   31   32   33  
item_id shop_id                           
0       54       0.0  0.0  0.0  0.0  0.0  
1       55       0.0  0.0  0.0  0.0  0.0  
2       54       0.0  0.0  0.0  0.0  0.0

In [21]:
dataset2 = pd.merge(test_df, dataset2, how = 'left', on=['item_id', 'shop_id']).fillna(0) #test data에 있는 id만 활용!
dataset2.drop(['shop_id', 'item_id', 'ID'], axis = 1, inplace = True)
dataset2.head()

/opt/conda/lib/python3.6/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


,"(item_price, 0)","(item_price, 1)","(item_price, 2)","(item_price, 3)","(item_price, 4)","(item_price, 5)","(item_price, 6)","(item_price, 7)","(item_price, 8)","(item_price, 9)","(item_price, 10)","(item_price, 11)","(item_price, 12)","(item_price, 13)","(item_price, 14)","(item_price, 15)","(item_price, 16)","(item_price, 17)","(item_price, 18)","(item_price, 19)","(item_price, 20)","(item_price, 21)","(item_price, 22)","(item_price, 23)","(item_price, 24)","(item_price, 25)","(item_price, 26)","(item_price, 27)","(item_price, 28)","(item_price, 29)","(item_price, 30)","(item_price, 31)","(item_price, 32)","(item_price, 33)"
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.008442,0.0,0.008442,0.006494,0.006494,0.0,0.0,0.0,0.004221,0.004870,0.00487,0.003247,0.002437,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.002922,0.001948,0.00000,0.001948,0.003247,0.003896
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.001948,0.000000,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000


## train test split

In [22]:
X_train = np.append(X_train,
         dataset2.values[:, :-1].reshape(X_train.shape[0], X_train.shape[1],1),
         axis = 2)

X_test = np.append(X_test,
                  dataset2.values[:, 1:].reshape(X_test.shape[0], X_test.shape[1], 1),
                  axis = 2)

print('X_train shape : ', X_train.shape)
print('X_test shape : ', X_test.shape)

X_train shape :  (214200, 33, 2)
X_test shape :  (214200, 33, 2)


## modeling

In [23]:
optimizer = RMSprop(lr = 0.005)

model_multi2 = Sequential()
model_multi2.add(LSTM(16, input_shape = (X_train.shape[1], X_train.shape[2]), return_sequences = True))
model_multi2.add(Dropout(0.3))
model_multi2.add(LSTM(32))
model_multi2.add(Dropout(0.3))
model_multi2.add(Dense(1))

model_multi2.compile(optimizer = optimizer, loss = 'mse', metrics = ['mse'])
model_multi2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 33, 16)            1216      
_________________________________________________________________
dropout_1 (Dropout)          (None, 33, 16)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                6272      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 7,521
Trainable params: 7,521
Non-trainable params: 0
_________________________________________________________________


In [24]:
history = model_multi2.fit(X_train, y_train ,batch_size=512, epochs = 30, verbose = 1,
                          callbacks = [early], validation_split=0.1)

Train on 192780 samples, validate on 21420 samples
Epoch 1/30
192780/192780 [==============================] - 40s 207us/step - loss: 33.2508 - mean_squared_error: 33.2508 - val_loss: 1.0018 - val_mean_squared_error: 1.0018
Epoch 2/30
192780/192780 [==============================] - 36s 188us/step - loss: 32.7201 - mean_squared_error: 32.7201 - val_loss: 0.9664 - val_mean_squared_error: 0.9664
Epoch 3/30
192780/192780 [==============================] - 37s 191us/step - loss: 32.4159 - mean_squared_error: 32.4159 - val_loss: 0.9480 - val_mean_squared_error: 0.9480
Epoch 4/30
192780/192780 [==============================] - 37s 190us/step - loss: 32.2045 - mean_squared_error: 32.2045 - val_loss: 0.9104 - val_mean_squared_error: 0.9104
Epoch 5/30
192780/192780 [==============================] - 36s 186us/step - loss: 31.9255 - mean_squared_error: 31.9255 - val_loss: 0.8742 - val_mean_squared_error: 0.8742
Epoch 6/30
192780/192780 [==============================] - 36s 184us/step - loss: 3

In [25]:
lstm_pred = model_multi2.predict(X_test).clip(0,20)

submission = pd.DataFrame({'ID': test_df['ID'], 'item_cnt_month': lstm_pred.ravel()})
submission.to_csv('20190730_lstm_add_item_price.csv',index=False)